In [1]:
import numpy as np
import torch
from causalbenchmark.novo.simulation.models import Bernoulli, ConditionalBernoulli, BernoulliNetwork

In [4]:
U = Bernoulli(0.1)
X = ConditionalBernoulli([U], [0.4, 0.6])
Y = ConditionalBernoulli([U, X], [[0.3, 0.5], [0.8, 0.2]])
net = BernoulliNetwork({'U': U, 'X': X, 'Y': Y})
print(net)

p(Y | U, X) p(X | U) p(U=0.1)


In [6]:
def ett(net):
	pU1 = net.marginals(X=1)['U'].item()
	pY00, pY01, pY10, pY11 = Y.param.view(-1).tolist()
	return pU1 * (pY01 - pY00) + (1 - pU1) * (pY11 - pY10)
def counferfactual(net):
	abduction = net.marginals(X=1)
	action_net = net.intervene(X=1, U=abduction['U'])
	return action_net.marginals()
def alt_ett(net):
	obs = net.marginals(X=1)
	cf = counferfactual(net)
	return obs['Y'] - cf['Y']

In [7]:
ett(net), alt_ett(net)

/home/fleeb/miniconda3/lib/python3.11/site-packages/torch/masked/maskedtensor/core.py:156: UserWarning: The PyTorch API of MaskedTensors is in prototype stage and will change in the near future. Please open a Github issue for features requests and see our documentation on the torch.masked module for further information about the project.
  warnings.warn(("The PyTorch API of MaskedTensors is in prototype stage "


(-0.48571428997176036, tensor(0.))

In [8]:
pU = net.marginals(X=1)['U']
pU

tensor(0.1429)

In [10]:
action_net = net.intervene(X=1, U=pU)
action_net.marginals()['Y']

tensor(0.4571)